In [1]:
import sys
sys.path.append('.')

import sedprofile.sedimentprofile as sp

import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2



In [2]:
filenameSuspD=r'C:\Users\cesposito\The Water Institute of the Gulf\P-00703_NSF_Caltech - General\Data\Tulane-LSU_2022Data\Isokinetic Suspended Sediment\MR173_Grain Size Compilation.xlsx'
filenameSuspC=r'C:\Users\cesposito\The Water Institute of the Gulf\P-00703_NSF_Caltech - General\Data\Tulane-LSU_2022Data\Isokinetic Suspended Sediment\MR173_Suspended Sediment Concentrations.xlsx'
filenameADCP=r'C:\Users\cesposito\The Water Institute of the Gulf\P-00703_NSF_Caltech - General\Data\Tulane-LSU_2022Data\StationaryADCP\MR173_StationaryADCP&Isokinetic.xlsx'

suspCconc=pd.read_excel(filenameSuspC,index_col=0,sheet_name='Sheet1')
suspCloc=pd.read_excel(filenameSuspC,sheet_name='IsoSamplingLocs',header=None)

ADCPData=pd.read_excel(filenameADCP)

rows_to_skip=list(range(0,2))+list(range(3,82))
suspD=pd.read_excel(filenameSuspD,index_col=None,sheet_name="Sheet1",skiprows=rows_to_skip,usecols="B:FY")     

channel_diam_lower=pd.read_excel(filenameSuspD,index_col=None,sheet_name="Sheet1",skiprows=rows_to_skip,usecols="A")     

In [3]:
aa=0.9

st1="HOP-1"
st2=str(aa)


search_string=st1+"_"+st2

myCols=suspD.filter(like=search_string)
print(myCols)



    HOP-1_0.9A  HOP-1_0.9B
0     0.014585    0.022548
1     0.025873    0.040180
2     0.038471    0.059365
3     0.055941    0.084798
4     0.072147    0.105926
..         ...         ...
88    0.000000    0.000000
89    0.000000    0.000000
90    0.000000    0.000000
91    0.000000    0.000000
92         NaN         NaN

[93 rows x 2 columns]


In [ ]:
usecols=lambda col: col != 0

print(usecols(0))

In [ ]:
test_dict["depth_frac"]


In [26]:

profile={}

#for each row in sheet IsoSamplingLocs (i.e. for each sampling location)
for i in range(1,suspCloc.shape[0]):
#for i in [5]:


    #loc = text in first column
    Location = suspCloc.at[i,0]

    #extract all rows of Sheet 1 with Location in column 2. These will become a single profile.
    filtered_rows = suspCconc[suspCconc["Station #"].fillna('').astype(str).str.contains(Location)]

    filtered_rows_ADCP = ADCPData[ADCPData["Station"].fillna('').astype(str).str.contains(Location)]

    filtered_rows.loc[:,"Total Sediment Conc (mg/l)"]

    profile[i] = {
        "station": Location,
        "QBelleChasse": -999,
        "source": "Not really sure",
        "susp_filepath": filenameSuspC,
        "datetime": filtered_rows_ADCP.iloc[0, filtered_rows_ADCP.columns.get_loc("Date & Time")],
        "lat_lon": [suspCloc.at[i,1],suspCloc.at[i,2]],
        "bed_depth_m": filtered_rows_ADCP.iloc[0, filtered_rows_ADCP.columns.get_loc("Total Depth (ft)")]*0.3048,
        "depth_frac": filtered_rows.loc[:,"Depths (0.1, 0.3, 0.5, 0.7 and 0.9H)"].values,
        "conc_sand": filtered_rows.loc[:,"Sand Conc (mg/l)"].values.T,
        "conc_mud": filtered_rows.loc[:,"Mud Conc (mg/l)"].values.T,
        "conc_total": filtered_rows.loc[:,"Total Sediment Conc (mg/l)"].values.T,
        "meas_channel_diam_lower": channel_diam_lower,
        "ADCP_filename": filtered_rows_ADCP.iloc[0, filtered_rows_ADCP.columns.get_loc("Stationary ADCP File Name")],
        "ADCP_ens_start": filtered_rows_ADCP.iloc[0, filtered_rows_ADCP.columns.get_loc("ADCP Ensemble Start")],
        "ADCP_ens_end": filtered_rows_ADCP.iloc[0, filtered_rows_ADCP.columns.get_loc("ADCP Ensemble End")],
    }

    for j in range(0,profile[i]["depth_frac"].shape[0]):
        search_string=profile[i]["station"]+"_"+str(profile[i]["depth_frac"][j])
        measCol=suspD.filter(like=search_string)

        if j==0:
            profile[i]["meas"]=measCol.values
        else:
                profile[i]["meas"]=np.c_[profile[i]["meas"],measCol.values]

    profile[i]["meas"]=profile[i]["meas"][:,0:6]    #this is a band aid, because there should be two 0.9 values but they keep getting duplicated

    profile[i]["depth"] = profile[i]["bed_depth_m"] * profile[i]["depth_frac"]    

    

profile[i]["meas"]         


array([], shape=(93, 0), dtype=float64)

In [42]:
profile[1]["meas"]

array([[2.26864e-02, 2.66431e-02, 3.12404e-02, 2.13672e-02, 4.56253e-02,
        1.97603e-02],
       [4.03824e-02, 4.74721e-02, 5.56678e-02, 3.80786e-02, 8.11586e-02,
        3.52322e-02],
       [5.95312e-02, 7.01206e-02, 8.23085e-02, 5.62507e-02, 1.19758e-01,
        5.21571e-02],
       [8.48238e-02, 1.00140e-01, 1.17757e-01, 8.03117e-02, 1.71149e-01,
        7.46930e-02],
       [1.05643e-01, 1.25070e-01, 1.47486e-01, 1.00241e-01, 2.14369e-01,
        9.36208e-02],
       [1.23828e-01, 1.47233e-01, 1.74239e-01, 1.17910e-01, 2.52998e-01,
        1.10753e-01],
       [1.40289e-01, 1.67709e-01, 1.99304e-01, 1.34172e-01, 2.88979e-01,
        1.26913e-01],
       [1.56223e-01, 1.87935e-01, 2.24409e-01, 1.50146e-01, 3.24882e-01,
        1.43176e-01],
       [1.69455e-01, 2.05241e-01, 2.46437e-01, 1.63657e-01, 3.56682e-01,
        1.57481e-01],
       [1.79732e-01, 2.19533e-01, 2.65326e-01, 1.74607e-01, 3.83923e-01,
        1.69855e-01],
       [1.87703e-01, 2.31570e-01, 2.81931e-01, 1.8

In [40]:
example_obj = sp.GrainSizeProfile.from_dict(profile[1])

example_obj

GrainSizeProfile(station='HOP-2', QBelleChasse=-999, source='Not really sure', susp_filepath='C:\\Users\\cesposito\\The Water Institute of the Gulf\\P-00703_NSF_Caltech - General\\Data\\Tulane-LSU_2022Data\\Isokinetic Suspended Sediment\\MR173_Suspended Sediment Concentrations.xlsx', datetime=Timestamp('2022-04-27 14:05:00'), lat_lon=[29.16966, -89.2573], bed_depth_m=17.6784, depth_frac=array([0.1, 0.3, 0.5, 0.7, 0.9, 0.9]), depth=array([ 1.76784,  5.30352,  8.8392 , 12.37488, 15.91056, 15.91056]), conc_sand=array([ 2.63157895,  9.36708861,  4.54545455,  8.47222222, 23.92857143,
       30.        ]), conc_mud=array([ 72.76315789, 102.53164557,  99.54545455, 116.52777778,
       129.04761905, 124.64285714]), conc_total=array([ 75.39473684, 111.89873418, 104.09090909, 125.        ,
       152.97619048, 154.64285714]), meas=array([[2.26864e-02, 2.66431e-02, 3.12404e-02, 2.13672e-02, 4.56253e-02,
        1.97603e-02],
       [4.03824e-02, 4.74721e-02, 5.56678e-02, 3.80786e-02, 8.11586e-02,

In [41]:
for attr in example_obj.__dict__:
    print(attr, example_obj.__dict__[attr])

station HOP-2
QBelleChasse -999
source Not really sure
susp_filepath C:\Users\cesposito\The Water Institute of the Gulf\P-00703_NSF_Caltech - General\Data\Tulane-LSU_2022Data\Isokinetic Suspended Sediment\MR173_Suspended Sediment Concentrations.xlsx
datetime 2022-04-27 14:05:00
lat_lon [29.16966, -89.2573]
bed_depth_m 17.6784
depth_frac [0.1 0.3 0.5 0.7 0.9 0.9]
depth [ 1.76784  5.30352  8.8392  12.37488 15.91056 15.91056]
conc_sand [ 2.63157895  9.36708861  4.54545455  8.47222222 23.92857143 30.        ]
conc_mud [ 72.76315789 102.53164557  99.54545455 116.52777778 129.04761905
 124.64285714]
conc_total [ 75.39473684 111.89873418 104.09090909 125.         152.97619048
 154.64285714]
meas [[2.26864e-02 2.66431e-02 3.12404e-02 2.13672e-02 4.56253e-02 1.97603e-02]
 [4.03824e-02 4.74721e-02 5.56678e-02 3.80786e-02 8.11586e-02 3.52322e-02]
 [5.95312e-02 7.01206e-02 8.23085e-02 5.62507e-02 1.19758e-01 5.21571e-02]
 [8.48238e-02 1.00140e-01 1.17757e-01 8.03117e-02 1.71149e-01 7.46930e-02]
 [

In [52]:
print(profile[1]["meas"])

[[2.26864e-02 2.66431e-02 3.12404e-02 2.13672e-02 4.56253e-02 1.97603e-02]
 [4.03824e-02 4.74721e-02 5.56678e-02 3.80786e-02 8.11586e-02 3.52322e-02]
 [5.95312e-02 7.01206e-02 8.23085e-02 5.62507e-02 1.19758e-01 5.21571e-02]
 [8.48238e-02 1.00140e-01 1.17757e-01 8.03117e-02 1.71149e-01 7.46930e-02]
 [1.05643e-01 1.25070e-01 1.47486e-01 1.00241e-01 2.14369e-01 9.36208e-02]
 [1.23828e-01 1.47233e-01 1.74239e-01 1.17910e-01 2.52998e-01 1.10753e-01]
 [1.40289e-01 1.67709e-01 1.99304e-01 1.34172e-01 2.88979e-01 1.26913e-01]
 [1.56223e-01 1.87935e-01 2.24409e-01 1.50146e-01 3.24882e-01 1.43176e-01]
 [1.69455e-01 2.05241e-01 2.46437e-01 1.63657e-01 3.56682e-01 1.57481e-01]
 [1.79732e-01 2.19533e-01 2.65326e-01 1.74607e-01 3.83923e-01 1.69855e-01]
 [1.87703e-01 2.31570e-01 2.81931e-01 1.83586e-01 4.07907e-01 1.80844e-01]
 [1.94051e-01 2.42167e-01 2.97183e-01 1.91183e-01 4.30104e-01 1.90998e-01]
 [1.99445e-01 2.51916e-01 3.11660e-01 1.97839e-01 4.51811e-01 2.00575e-01]
 [2.03291e-01 2.60161e-01